In [1]:
import pandas as pd
import urllib.request as ur
import os
import zipfile

pd.options.display.max_rows = 3000

temp_files = []

dfs = {}
state_dfs = {}

DROP_AKHI = True

In [2]:
census = {1:'ne', 2:'ma', 3:'enc', 4:'wnc', 5:'sa', 6:'esc', 7:'wsc', 8:'mt', 9:'pac'}

rucc = {-1: 'null', 0:'central_metro_1m', 1:'fringe_metro_1m', 2:'met_over_250k', 3:'met_sub_250k',
        4:'urban_over_20k_adj', 5:'urban_over_20k_notadj', 6:'urban_under_20k_adj', 7:'urban_under_20k_notadj',
        8:'rural_adj', 9:'rural_notadj'}

uic = {2013:{1:'In large metro area of 1+ million residents',
             2:'In small metro area of less than 1 million residents',
             3:'Micropolitan area adjacent to large metro area',
             4:'Noncore adjacent to large metro area',
             5:'Micropolitan area adjacent to small metro area',
             6:'Noncore adjacent to small metro area and contains a town of at least 2,500 residents',
             7:'Noncore adjacent to small metro area and does not contain a town of at least 2,500 residents',
             8:'Micropolitan area not adjacent to a metro area',
             9:'Noncore adjacent to micro area and contains a town of at least 2,500 residents',
             10:'Noncore adjacent to micro area and does not contain a town of at least 2,500 residents',
             11:'Noncore not adjacent to metro or micro area and contains a town of at least 2,500 residents',
             12:'Noncore not adjacent to metro or micro area and does not contain a town of at least 2,500 residents'},
       2003:{1:'In large metro area of 1+ million residents',
             2:'In small metro area of less than 1 million residents',
             3:'Micropolitan area adjacent to large metro area',
             4:'Noncore adjacent to large metro area',
             5:'Micropolitan area adjacent to small metro area',
             6:'Noncore adjacent to small metro area and contains a town of at least 2,500 residents',
             7:'Noncore adjacent to small metro area and does not contain a town of at least 2,500 residents',
             8:'Micropolitan area not adjacent to a metro area',
             9:'Noncore adjacent to micro area and contains a town of at least 2,500 residents',
             10:'Noncore adjacent to micro area and does not contain a town of at least 2,500 residents',
             11:'Noncore not adjacent to metro or micro area and contains a town of at least 2,500 residents',
             12:'Noncore not adjacent to metro or micro area and does not contain a town of at least 2,500 residents'},
       1993:{1:'Large-in a metro area with at least 1 million residents or more',
             2:'Small-in a metro area with fewer than 1 million residents',
             3:'Adjacent to a large metro area and contains a city of at least 10,000 residents',
             4:'Adjacent to a large metro area and does not have a city of at least 10,000 residents',
             5:'Adjacent to a small metro area and contains a city of at least 10,000 residents',
             6:'Adjacent to a small metro area and does not have a city of at least 10,000 residents',
             7:'Not adjacent to a metro area and contains a city of at least 10,000 residents',
             8:'Not adjacent to a metro area and contains a town of 2,500- 9,999 residents',
             9:'Not adjacent to a metro area and does not contain a town of at least 2,500 residents'}}

topo_codes = {1:'flat_plains', 2:'smooth_plains', 3:'irr_plains_relief', 4:'irr_plains',
              5:'table_moderate', 6:'table_considerable', 7:'table_high', 8:'table_vhigh',
              9:'plains_hills', 10:'plains_hhills', 11:'plains_lmountains', 12:'plains_hmountains',
              13:'open_lhills', 14:'open_hills', 15:'open_hhills', 16:'open_lmountains', 17:'open_hmountains',
              18:'hills', 19:'hhills', 20:'lmountains', 21:'hmountains'}

**FIPS code data**

In [3]:
df_file = "fips.csv"
if not os.path.isfile(df_file):
    ur.urlretrieve("https://www2.census.gov/geo/docs/reference/codes/files/national_county.txt", df_file)
name_df = pd.read_csv(df_file, header=None)
temp_files.append(df_file)

# Rename columns
name_df.columns = ['state', 'state_fips', 'county_fips', 'county', 'fips_class']

# Eliminate AK & HI:
if DROP_AKHI:
    name_df = name_df[name_df.state != 'AK']
    name_df = name_df[name_df.state != 'HI']

# Eliminate non-state counties
for state in ['AS', 'GU', 'MP', 'PR', 'UM', 'VI']:
    name_df = name_df[name_df.state != state]

# Create FIPS columns
name_df['FIPS'] = name_df.state_fips*1000 + name_df.county_fips

# Set index to FIPS
name_df.set_index('FIPS', inplace=True)

# Drop unneeded columns
name_df.drop(['state_fips', 'county_fips', 'fips_class'], axis=1, inplace=True)


dfs['name'] = name_df
# name_df

** Counties Gazetteer File **

In [4]:
df_file = "Gaz_counties_national.txt"
if not os.path.isfile(df_file):
    if not os.path.isfile("Gaz_counties_national.zip"):
        ur.urlretrieve("http://www2.census.gov/geo/docs/maps-data/data/gazetteer/Gaz_counties_national.zip",
                       "Gaz_counties_national.zip")
        with zipfile.ZipFile("Gaz_counties_national.zip", "r") as zip_ref:
            zip_ref.extractall('.')
temp_files.append("Gaz_counties_national.zip")
gaz_df = pd.read_table(df_file, encoding='Latin1')
temp_files.append(df_file)

# Eliminate AK & HI:
if DROP_AKHI:
    gaz_df = gaz_df[gaz_df.USPS != 'AK']
    gaz_df = gaz_df[gaz_df.USPS != 'HI']
# Eliminate PR
gaz_df = gaz_df[gaz_df.USPS != 'PR']

# Drop unneeded columns
gaz_df.drop(['USPS', 'ANSICODE', 'NAME', 'POP10', 'ALAND', 'AWATER'], axis=1, inplace=True)

# Rename columns
gaz_df.columns = ['FIPS', 'housing_units', 'area_land', 'area_water', 'lat', 'long']

# Set FIPS as index
gaz_df.set_index(['FIPS'], inplace=True)


dfs['gaz'] = gaz_df
# gaz_df

** Natural Amenities **

In [5]:
df_file = "natamen"
if not os.path.isfile(df_file):
    ur.urlretrieve("https://www.ers.usda.gov/webdocs/DataFiles/52201/natamenf_1_.xls", df_file)
natamen_df = pd.read_excel(df_file, skiprows=104)
temp_files.append(df_file)

natamen_df.columns = ['FIPS', 'fips2', 'state', 'county', 'census_div', 'rucc_1993', 'uic_1993', 'jan_temp_mean',
                      'jan_sun_mean', 'jul_temp_mean', 'jul_hum_mean', 'topo_code', 'pct_water', 'log_pct_water',
                      'jan_temp_z', 'jan_sun_z', 'jul_temp_z', 'jul_hum_z', 'topo_z', 'water_z', 'nat_amen_scale',
                      'nat_amen_rank']

# Drop AK & HI
if DROP_AKHI:
    natamen_df = natamen_df[natamen_df['state'] != 'AK']
    natamen_df = natamen_df[natamen_df['state'] != 'HI']

# Set index to FIPS
natamen_df.set_index(['FIPS'], inplace=True)

# Set to ints in some cases
for col in ['census_div', 'rucc_1993', 'uic_1993', 'topo_code', 'nat_amen_rank']:
    natamen_df[col] = natamen_df[col].astype(int)

# Drop unneeded columns
natamen_df.drop(['fips2', 'state', 'county', 'uic_1993', 'rucc_1993'], axis=1, inplace=True)

# Give coded values better names
#natamen_df['census_div'] = natamen_df['census_div'].map(census)
#natamen_df['topo_code'] = natamen_df['topo_code'].map(topo_codes)


dfs['natural'] = natamen_df
# natamen_df

** Rural-Urban Continuum Codes ** (1974, 1983, 1993, 2003, 2013)

In [6]:
rucc_df = pd.DataFrame(index=name_df.index)

for df_file, colnum, year in [("ruralurbancodes1974.xls", 3, 1974), ("cd8393.xls", 3, 1983),
                              ("code93.xls", 3, 1993), ("ruralurbancodes2003.xls", 4, 2003),
                              ("ruralurbancodes2013.xls", 4, 2013)]:
    if not os.path.isfile(df_file):
        ur.urlretrieve("https://www.ers.usda.gov/webdocs/DataFiles/53251/"+df_file, df_file)
    rucc_part_df = pd.read_excel(df_file)
    temp_files.append(df_file)

    rucc_part_df = rucc_part_df.iloc[:,[0, colnum]]
    rucc_part_df.columns = ['FIPS', 'RUCC_'+str(year)]
    rucc_part_df.dropna(axis=0, inplace=True)
    rucc_part_df = rucc_part_df.astype(int)
    rucc_part_df.set_index('FIPS', inplace=True)

    rucc_df = rucc_df.join(rucc_part_df)

# Fill NaNs with -1
rucc_df.fillna(-1, axis=0, inplace=True)
rucc_df = rucc_df.astype(int)

# Give coded values better names
#for col in rucc_df.columns:
#    rucc_df[col] = rucc_df[col].map(rucc)


dfs['rucc'] = rucc_df
# rucc_df

** Urban Influence Codes ** (1993, 2003, 2013)

In [7]:
uic_df = pd.DataFrame(index=name_df.index)

for df_file, colnum, year, sheet in [("UrbanInfluenceCodes.xls", 7, 1993, "Urban Influence Codes"),
                                     ("UrbanInfluenceCodes.xls", 3, 2003, "Urban Influence Codes"),
                                     ("UrbanInfluenceCodes2013.xls", 4, 2013, "Urban Influence Codes 2013")]:
    if not os.path.isfile(df_file):
        ur.urlretrieve("https://www.ers.usda.gov/webdocs/DataFiles/53797/"+df_file, df_file)
    uic_part_df = pd.read_excel(df_file, sheetname=sheet)
    temp_files.append(df_file)

    uic_part_df = uic_part_df.iloc[:,[0, colnum]]
    uic_part_df.columns = ['FIPS', 'UIC_'+str(year)]
    uic_part_df.dropna(axis=0, inplace=True)
    uic_part_df = uic_part_df.astype(int)
    uic_part_df.set_index('FIPS', inplace=True)

    # Give coded values better names
    #for col in uic_part_df.columns:
    #    uic_part_df[col] = uic_part_df[col].map(uic[year])
    
    uic_df = uic_df.join(uic_part_df)


dfs['uic'] = uic_df
#uic_df

**2016 Presidential Election data**

In [8]:
df_file = "2016_US_County_Level_Presidential_Results.csv"
if not os.path.isfile(df_file):
    ur.urlretrieve("https://raw.githubusercontent.com/tonmcg/County_Level_Election_Results_12-16/master/" +
                       df_file, df_file)
election_df = pd.read_csv(df_file)
temp_files.append(df_file)

# # Alaska isn't accurate to the borough level, so drop the boroughs & reassign the FIPS
# election_df.drop(range(28), axis=0, inplace=True)
# election_df.set_value(28, 'combined_fips', 2000)
# Alaska isn't accurate to the borough level, but let's just drop AK & HI to keep it continental
if DROP_AKHI:
    election_df = election_df[election_df.state_abbr != 'AK']
    election_df = election_df[election_df.state_abbr != 'HI']

# Drop the index column, set combined_fips to be index, and rename it FIPS
election_df.drop(election_df.columns[0], axis=1, inplace=True)
election_df.set_index(['combined_fips'], inplace=True)
election_df.index.names = ['FIPS']

# convert diff to an integer
election_df['diff'] = election_df['diff'].apply(lambda d: int(d.replace(',', '')))
# convert per_point_diff to a float
election_df['per_point_diff'] = election_df['per_point_diff'].apply(lambda ppd: float(ppd.replace('%', ''))/100)
#convert votes to integers
election_df['votes_dem'] = election_df['votes_dem'].astype(int)
election_df['votes_gop'] = election_df['votes_gop'].astype(int)
election_df['total_votes'] = election_df['total_votes'].astype(int)

# Drop unneeded columns
election_df.drop(['state_abbr', 'county_name'], axis=1, inplace=True)

# Drop differences
election_df.drop(['diff', 'per_point_diff'], axis=1, inplace=True)

# Rename columns
election_df.columns = ['votes_clinton', 'votes_trump', 'votes_total', 'pct_clinton', 'pct_trump']


dfs['election'] = election_df
# election_df

**2016 Census data**

Codebook: https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2010-2016/cc-est2016-alldata.pdf

In [9]:
df_file = "cc-est2016-alldata.csv"
if not os.path.isfile(df_file):
    ur.urlretrieve("https://www2.census.gov/programs-surveys/popest/datasets/2010-2016/counties/asrh/" +
                       df_file, df_file)
census_df = pd.read_csv(df_file, encoding="Latin1")
temp_files.append(df_file)

# Drop state-level data
census_df = census_df[census_df.SUMLEV != 40]

# Combine STATE & COUNTY as FIPS
census_df['FIPS'] = census_df['STATE']*1000 + census_df['COUNTY']

# Drop AK & HI
if DROP_AKHI:
    census_df = census_df[census_df.STATE != 2]
    census_df = census_df[census_df.STATE != 15]

# Drop all rows other than the 7/1/2016 population estimate
census_df = census_df[census_df.YEAR == 9]

# Drop in-combination races (except "Two or More Races") and
# all "Hispanic ..." and "Not Hispanic ..." ethnicities cross with race (but keep Hispanic male/female itself)
census_df.drop(['WAC_MALE', 'BAC_MALE', 'IAC_MALE', 'AAC_MALE', 'NAC_MALE',
                'NH_MALE', 'NHWA_MALE', 'NHBA_MALE', 'NHIA_MALE', 'NHAA_MALE', 'NHNA_MALE',
                'NHTOM_MALE', 'NHWAC_MALE', 'NHBAC_MALE', 'NHIAC_MALE', 'NHAAC_MALE', 'NHNAC_MALE',
                'HWA_MALE', 'HBA_MALE', 'HIA_MALE', 'HAA_MALE', 'HNA_MALE',
                'HTOM_MALE', 'HWAC_MALE', 'HBAC_MALE', 'HIAC_MALE', 'HAAC_MALE', 'HNAC_MALE',
                'WAC_FEMALE', 'BAC_FEMALE', 'IAC_FEMALE', 'AAC_FEMALE', 'NAC_FEMALE',
                'NH_FEMALE', 'NHWA_FEMALE', 'NHBA_FEMALE', 'NHIA_FEMALE', 'NHAA_FEMALE', 'NHNA_FEMALE',
                'NHTOM_FEMALE', 'NHWAC_FEMALE', 'NHBAC_FEMALE', 'NHIAC_FEMALE', 'NHAAC_FEMALE', 'NHNAC_FEMALE',
                'HWA_FEMALE', 'HBA_FEMALE', 'HIA_FEMALE', 'HAA_FEMALE', 'HNA_FEMALE',
                'HTOM_FEMALE', 'HWAC_FEMALE', 'HBAC_FEMALE', 'HIAC_FEMALE', 'HAAC_FEMALE', 'HNAC_FEMALE'],
              axis=1, inplace=True)

# Set FIPS as index
census_df.set_index(['FIPS'], inplace=True)

# Extract age data crossed with sex to temp_df
temp_df = pd.DataFrame()
age_cols = ['pop_0to4', 'pop_5to9', 'pop_10to14', 'pop_15to19', 'pop_20to24', 'pop_25to29', 'pop_30to34',
            'pop_35to39', 'pop_40to44', 'pop_45to49', 'pop_50to54', 'pop_55to59', 'pop_60to64', 'pop_65to69',
            'pop_70to74', 'pop_75to79', 'pop_80to84', 'pop_85plus']
for age_id, age_name in zip(range(1,19), age_cols):
    #print(age_id, age_name)
    temp_df[age_name] = census_df[census_df.AGEGRP == age_id].TOT_POP
    temp_df[age_name+'_male'] = census_df[census_df.AGEGRP == age_id].TOT_MALE
    temp_df[age_name+'_female'] = census_df[census_df.AGEGRP == age_id].TOT_FEMALE

# Drop age groups other than the total
census_df = census_df[census_df.AGEGRP == 0]
    
# Drop unused columns
census_df.drop(['SUMLEV', 'STATE', 'COUNTY', 'STNAME', 'CTYNAME', 'YEAR', 'AGEGRP'], axis=1, inplace=True)

# Rename columns
census_df.columns = ['pop', 'pop_male', 'pop_female', 'white_male', 'white_female',
                     'black_male', 'black_female', 'amerindian_male', 'amerindian_female',
                     'asian_male', 'asian_female', 'pacific_male', 'pacific_female',
                     'multiracial_male', 'multiracial_female', 'hispanic_male', 'hispanic_female']

# Join with and delete temp_df
census_df = census_df.join(temp_df)
del temp_df


dfs['census']= census_df
# census_df

** Religious Congregations and Membership Study of the Association of Religion Data Archives (ARDA) **

In [10]:
df_file = "rcms2010.xlsx"
if not os.path.isfile(df_file):
    ur.urlretrieve("http://www.thearda.com/download/download.aspx?file=U.S.%20Religion%20Census%20Religious%20Congregations%20and%20Membership%20Study,%202010%20(County%20File).XLSX",
                   df_file)
rcms_df = pd.read_excel(df_file)
temp_files.append(df_file)

# Fill NaNs with 0
rcms_df.fillna(0, inplace=True)

# Drop AK & HI
if DROP_AKHI:
    rcms_df = rcms_df[rcms_df['STABBR'] != 'AK']
    rcms_df = rcms_df[rcms_df['STABBR'] != 'HI']

# Set index to FIPS
rcms_df.set_index(['FIPS'], inplace=True)

# Set to ints in some cases
# for col in ['census_div', 'rucc_1993', 'uic_1993', 'topo_code', 'nat_amen_rank']:
#     rcms_df[col] = rcms_df[col].astype(int)

# Drop unneeded columns
rcms_df.drop(['STCODE', 'STABBR', 'STNAME', 'CNTYCODE', 'CNTYNAME', 'POP2010'], axis=1, inplace=True)

# Rename columns
def rename_rcms(col):
    col = col.lower()
    if col.endswith('cng'):
        col = 'congregations_'+col[:-3]
    elif col.endswith('adh'):
        col = 'adherents_'+col[:-3]
    elif col.endswith('rate'):
        col = 'pct_'+col[:-4]
    return 'RCMS_'+col
rcms_df.columns = [rename_rcms(col) for col in rcms_df.columns]

# Change rates to percentages and convert others to int
for col in rcms_df.columns:
    if '_pct_' in col:
        rcms_df[col] = rcms_df[col]/1000
    else:
        rcms_df[col] = rcms_df[col].astype(int)

dfs['rcms'] = rcms_df
# rcms_df

**2015 OSADI - Beneficiary data**

In [11]:
df_file = "oasdi_2015.xlsx"
if not os.path.isfile(df_file):
    ur.urlretrieve("https://www.ssa.gov/policy/docs/statcomps/oasdi_sc/2015/tables4-5alt.xlsx",
                   df_file)
oasdi_ben_df = pd.read_excel(df_file, sheetname="Table 4", skiprows=4, header=None)
temp_files.append(df_file)

# Rename columns
oasdi_ben_df.columns = ['state', 'drop', 'county', 'FIPS', 'total',
                        'retirement_workers', 'retirement_spouses', 'retirement_children',
                        'survivors_widows_parents', 'survivors_children',
                        'disability_workers', 'disability_spouses', 'disability_children',
                        'age65_M', 'age65_F']

# Drop extra column
oasdi_ben_df.drop(['drop'], axis=1, inplace=True)

# Drop non-state rows
oasdi_ben_df = oasdi_ben_df[:oasdi_ben_df[oasdi_ben_df.state == 'Outlying areas'].index[0]]

# Change numbers to integers
for col in ['FIPS', 'total', 'retirement_workers', 'retirement_spouses', 'retirement_children',
            'survivors_widows_parents', 'survivors_children', 'disability_workers', 'disability_spouses',
            'disability_children', 'age65_M', 'age65_F']:
    oasdi_ben_df[col] = oasdi_ben_df[col].astype(int)

# Drop AK & HI
if DROP_AKHI:
    oasdi_ben_df = oasdi_ben_df[oasdi_ben_df.state != 'Alaska']
    oasdi_ben_df = oasdi_ben_df[oasdi_ben_df.state != 'Hawaii']

# Drop unneeded columns
oasdi_ben_df.drop(['state', 'county'], axis=1, inplace=True)

# Set FIPS as index
oasdi_ben_df.set_index(['FIPS'], inplace=True)

# Rename columns
oasdi_ben_df.columns = ['OASDI_ben_'+col for col in oasdi_ben_df.columns]


dfs['oasdi_ben'] = oasdi_ben_df
# oasdi_ben_df

**2015 OSADI - Payments data**

In [12]:
df_file = "oasdi_2015.xlsx"
if not os.path.isfile(df_file):
    ur.urlretrieve("https://www.ssa.gov/policy/docs/statcomps/oasdi_sc/2015/tables4-5alt.xlsx",
                   df_file)
oasdi_pay_df = pd.read_excel(df_file, sheetname="Table 5", skiprows=4, header=None)
temp_files.append(df_file)

# Rename columns
oasdi_pay_df.columns = ['state', 'drop', 'county', 'FIPS', 'total',
                        'retirement_workers', 'retirement_spouses', 'retirement_children',
                        'survivors_widows_parents', 'survivors_children',
                        'disability_workers', 'disability_spouses', 'disability_children',
                        'age65_M', 'age65_F']

# Drop extra column
oasdi_pay_df.drop(['drop'], axis=1, inplace=True)

# Drop non-county rows
oasdi_pay_df = oasdi_pay_df[:oasdi_pay_df[oasdi_pay_df.state == 'Outlying areas'].index[0]]

# Change numbers to integers
for col in ['FIPS', 'total', 'retirement_workers', 'retirement_spouses', 'retirement_children',
            'survivors_widows_parents', 'survivors_children', 'disability_workers', 'disability_spouses',
            'disability_children', 'age65_M', 'age65_F']:
    oasdi_pay_df[col] = oasdi_pay_df[col].astype(int)
    #oasdi_pay_df[col] *= 1000

# Drop AK & HI
if DROP_AKHI:
    oasdi_pay_df = oasdi_pay_df[oasdi_pay_df.state != 'Alaska']
    oasdi_pay_df = oasdi_pay_df[oasdi_pay_df.state != 'Hawaii']

# Drop unneeded columns
oasdi_pay_df.drop(['state', 'county'], axis=1, inplace=True)

# Set FIPS as index
oasdi_pay_df.set_index(['FIPS'], inplace=True)

# Rename columns
oasdi_pay_df.columns = ['OASDI_pay_'+col for col in oasdi_pay_df.columns]


dfs['oasdi_pay'] = oasdi_pay_df
# oasdi_pay_df

**2015 SAIPE data**

In [13]:
df_file = "saipe_2015.xls"
if not os.path.isfile(df_file):
    ur.urlretrieve("https://www.census.gov/did/www/saipe/downloads/estmod15/est15ALL.xls",
                   df_file)
saipe_df = pd.read_excel(df_file, skiprows=4, header=None)
temp_files.append(df_file)

# Rename columns
saipe_df.columns = ['state_FIPS', 'county_FIPS', 'state', 'county',
                    'poverty_all', 'ci_lower_all', 'ci_upper_all',
                    'poverty_pct_all', 'poverty_pct_ci_lower_all', 'poverty_pct_ci_upper_all',
                    'poverty_0to17', 'ci_lower_0to17', 'ci_upper_0to17',
                    'poverty_pct_0to17', 'poverty_pct_ci_lower_0to17', 'poverty_pct_ci_upper_0to17',
                    'poverty_5to17', 'ci_lower_5to17', 'ci_upper_5to17',
                    'poverty_pct_5to17', 'poverty_pct_ci_lower_5to17', 'poverty_pct_ci_upper_5to17',
                    'median_hhi', 'median_hhi_ci_lower', 'median_hhi_ci_upper',
                    'poverty_0to4', 'ci_lower_0to4', 'ci_upper_0to4',
                    'poverty_pct_0to4', 'poverty_pct_ci_lower_0to4', 'poverty_pct_ci_upper_0to4']

# Drop Ages 0-4 (which is state-level only)
saipe_df.drop(['poverty_0to4', 'ci_lower_0to4', 'ci_upper_0to4', 'poverty_pct_0to4',
               'poverty_pct_ci_lower_0to4', 'poverty_pct_ci_upper_0to4'], axis=1, inplace=True)

# Drop non-county rows
saipe_df = saipe_df[saipe_df.county_FIPS != 0]

# Drop AK & HI
if DROP_AKHI:
    saipe_df = saipe_df[saipe_df.state_FIPS != 2]
    saipe_df = saipe_df[saipe_df.state_FIPS != 15]

# Combine state_FIPS & county_FIPS as FIPS
saipe_df['FIPS'] = saipe_df['state_FIPS']*1000 + saipe_df['county_FIPS']
saipe_df.drop(['state_FIPS', 'county_FIPS'], axis=1, inplace=True)

# Set index to FIPS
saipe_df.set_index(['FIPS'], inplace=True)

# Convert percentages to range [0,1]
for col in saipe_df.columns:
    if 'pct' in col:
        saipe_df[col] /= 100

# Convert 0 to 17 into 0 to 4:
saipe_df.poverty_0to17 -= saipe_df.poverty_5to17

# Drop unneeded columns
saipe_df.drop(['state', 'county'], axis=1, inplace=True)

# Drop CIs
saipe_df.drop(['ci_lower_all', 'ci_upper_all', 'poverty_pct_ci_lower_all',
               'poverty_pct_ci_upper_all', 'ci_lower_0to17', 'ci_upper_0to17',
               'poverty_pct_ci_lower_0to17', 'poverty_pct_ci_upper_0to17', 'ci_lower_5to17', 'ci_upper_5to17',
               'poverty_pct_ci_lower_5to17', 'poverty_pct_ci_upper_5to17',
               'median_hhi_ci_lower', 'median_hhi_ci_upper'], axis=1, inplace=True)

# Drop percentages
# saipe_df.drop(['poverty_pct_all', 'poverty_pct_0to17', 'poverty_pct_5to17'], axis=1, inplace=True)

# Rename columns
saipe_df.columns = ['SAIPE_'+_ for _ in ['poverty', 'poverty_0to4', 'poverty_5to17',
                                         'pct_poverty_all', 'pct_poverty_0to4', 'pct_poverty_5to17',
                                         'median_hhi']]


dfs['saipe'] = saipe_df
# saipe_df

**2015 SSI data**

In [14]:
df_file = "ssi_2015.xlsx"
if not os.path.isfile(df_file):
    ur.urlretrieve("https://www.ssa.gov/policy/docs/statcomps/ssi_sc/2015/table03alt.xlsx",
                   df_file)
ssi_df = pd.read_excel(df_file, skiprows=3, header=None, na_values='(X)')
temp_files.append(df_file)

# Rename columns
ssi_df.columns = ['state', 'drop', 'county', 'FIPS', 'total', 'aged', 'blind_disabled',
                  'age_0to17', 'age_18to64', 'age_65plus', 'alsoOASDI', 'payments']

# Drop extra column
ssi_df.drop(['drop'], axis=1, inplace=True)

# Drop non-county rows
ssi_df = ssi_df[:ssi_df[ssi_df.state == 'Outlying areas'].index[0]]

# Change FIPS to int
ssi_df['FIPS'] = ssi_df['FIPS'].astype(int)

# Drop AK & HI
if DROP_AKHI:
    ssi_df = ssi_df[ssi_df.state != 'Alaska']
    ssi_df = ssi_df[ssi_df.state != 'Hawaii']

# Set FIPS as index
ssi_df.set_index(['FIPS'], inplace=True)

# Drop unneeded columns
ssi_df.drop(['state', 'county'], axis=1, inplace=True)

# Rename columns
ssi_df.columns = ['SSI_'+_ for _ in ['total', 'aged', 'blind_disabled', 'age_0to17', 'age_18to64', 'age_65plus',
                                     'and_OASDI', 'pay']]


dfs['ssi'] = ssi_df
# ssi_df

** 2015 Veteran Health Administration (VHA) enrollees **

In [15]:
df_file = "VAEnrolleesByCountyFY2015.xlsx"
if not os.path.isfile(df_file):
    ur.urlretrieve("https://raw.githubusercontent.com/vacobrydsk/VAEnrolleesByCountyFY2015/master/" + df_file,
                   df_file)
vha_df = pd.read_excel(df_file, skiprows=2, header=None)
temp_files.append(df_file)

# Rename columns
vha_df.columns = ['state', 'state_fips', 'county', 'county_fips', 'fips_class', 'FIPS', 'VHA_enrollees']

# Drop non-county rows
#vha_df = vha_df[:vha_df[vha_df.state == 'Outlying areas'].index[0]]

# Change FIPS to int
vha_df['FIPS'] = vha_df['FIPS'].astype(int)

# Drop AK & HI
if DROP_AKHI:
    vha_df = vha_df[vha_df.state != 'AK']
    vha_df = vha_df[vha_df.state != 'HI']

# Drop VI, PR
for terr in ['VI', 'PR', 'GU']:
    vha_df = vha_df[vha_df.state != terr]

    
# Set FIPS as index
vha_df.set_index(['FIPS'], inplace=True)

# Drop unneeded columns
vha_df.drop(['state', 'state_fips', 'county', 'county_fips', 'fips_class'], axis=1, inplace=True)


dfs['vha'] = vha_df
# vha_df

Data from CDC WONDER, Underlying Cause of Death Detailed Mortality database:
* grouped by county
* including 95% CI and SE on the crude rate, age adjusted rate (+ 95% CI & SE), and percent of total deaths
* all states, ages, genders, races, ethnicities, years (1999-2015), days, places, etc
* drug-induced causes ONLY
* including totals, zero values, & suppressed values, 2 decimal places of precision

https://wonder.cdc.gov/controller/saved/D76/D16F414

Also, the same, but:
* all causes of death

https://wonder.cdc.gov/controller/saved/D76/D16F716

Also, the same, but:
* grouped by state

https://wonder.cdc.gov/controller/saved/D76/D16F415

Also, the same, but:
* grouped by state
* all causes of death

https://wonder.cdc.gov/controller/saved/D76/D16F717

To retrieve the data:
1. go the the URL
2. click "I Agree"
3. click "Send"
4. rename the files "dm_county_drug.tsv", "dm_county_all.tsv", "dm_state_drug.tsv", and
   "dm_state_all.tsv" respectively

In [16]:
df_file = "dm_county_drug.tsv"
dmd_df = pd.read_table(df_file, na_values=['Suppressed', 'Unreliable'], skipfooter=129, engine='python')

# Drop first two cols
dmd_df.drop(['Notes', 'County'], axis=1, inplace=True)

# Rename columns
dmd_df.columns = ['FIPS', 'deaths', 'population', 'crude', 'crude_CI_lower', 'crude_CI_upper', 'crude_se',
                  'age_adj', 'age_adj_CI_lower', 'age_adj_CI_upper', 'age_adj_se', 'pct_deaths']

# Convert percentage to [0, 1]
dmd_df.pct_deaths = dmd_df.pct_deaths.apply(lambda p: float(str(p).replace('%', ''))/100)

# Add the state FIPS
dmd_df['state_FIPS'] = dmd_df.FIPS // 1000

# Drop AK & HI
if DROP_AKHI:
    dmd_df = dmd_df[dmd_df.state_FIPS != 2]
    dmd_df = dmd_df[dmd_df.state_FIPS != 15]

# Set FIPS as index
dmd_df.set_index(['FIPS'], inplace=True)

# Drop unneeded columns (incl. CI, SE, & percent)
dmd_df.drop(['population', 'crude_CI_lower', 'crude_CI_upper', 'crude_se',
             'age_adj_CI_lower', 'age_adj_CI_upper', 'age_adj_se', 'pct_deaths'], axis=1, inplace=True)

# Rename columns
dmd_df.columns = ['DM_deaths_drug', 'DM_crude_rate_drug', 'DM_age_adj_rate_drug', 'state_FIPS']

# dfs['dm_drug'] = dmd_df
# dmd_df

In [17]:
df_file = "dm_county_all.tsv"
dma_df = pd.read_table(df_file, na_values=['Suppressed', 'Unreliable'], skipfooter=124, engine='python')

# Drop first two cols
dma_df.drop(['Notes', 'County'], axis=1, inplace=True)

# Rename columns
dma_df.columns = ['FIPS', 'deaths', 'population', 'crude', 'crude_CI_lower', 'crude_CI_upper', 'crude_se',
                  'age_adj', 'age_adj_CI_lower', 'age_adj_CI_upper', 'age_adj_se']

# Add the state FIPS
dma_df['state_FIPS'] = dma_df.FIPS // 1000

# Drop AK & HI
if DROP_AKHI:
    dma_df = dma_df[dma_df.state_FIPS != 2]
    dma_df = dma_df[dma_df.state_FIPS != 15]

# Set FIPS as index
dma_df.set_index(['FIPS'], inplace=True)

# Drop unneeded columns (incl. CI, SE, & percent)
dma_df.drop(['population', 'crude_CI_lower', 'crude_CI_upper', 'crude_se',
             'age_adj_CI_lower', 'age_adj_CI_upper', 'age_adj_se'], axis=1, inplace=True)

# Rename columns
dma_df.columns = ['DM_deaths_all', 'DM_crude_rate_all', 'DM_age_adj_rate_all', 'state_FIPS']

# dfs['dm_all'] = dma_df
# dma_df

In [18]:
df_file = "dm_state_drug.tsv"
dmdst_df = pd.read_table(df_file, na_values=['Suppressed', 'Unreliable'], skipfooter=70, engine='python')

# Drop first two cols
dmdst_df.drop(['Notes', 'State'], axis=1, inplace=True)

# Rename columns
dmdst_df.columns = ['state_FIPS', 'deaths', 'population', 'crude', 'crude_CI_lower', 'crude_CI_upper', 'crude_se',
                    'age_adj', 'age_adj_CI_lower', 'age_adj_CI_upper', 'age_adj_se', 'pct_deaths']

# Convert percentage to [0, 1]
dmdst_df.pct_deaths = dmdst_df.pct_deaths.apply(lambda p: float(str(p).replace('%', ''))/100)

# Drop AK & HI
if DROP_AKHI:
    dmdst_df = dmdst_df[dmdst_df.state_FIPS != 2]
    dmdst_df = dmdst_df[dmdst_df.state_FIPS != 15]

# Set FIPS as index
dmdst_df.set_index(['state_FIPS'], inplace=True)

# Drop unneeded columns (incl. CI, SE, & percent)
dmdst_df.drop(['population', 'crude_CI_lower', 'crude_CI_upper', 'crude_se',
               'age_adj_CI_lower', 'age_adj_CI_upper', 'age_adj_se', 'pct_deaths'], axis=1, inplace=True)

# Rename columns
dmdst_df.columns = ['DM_deaths_drug', 'DM_crude_rate_drug', 'DM_age_adj_rate_drug']


state_dfs['dm_drug'] = dmdst_df
# dmdst_df

In [19]:
df_file = "dm_state_all.tsv"
dmast_df = pd.read_table(df_file, na_values=['Suppressed', 'Unreliable'], skipfooter=60, engine='python')

# Drop first two cols
dmast_df.drop(['Notes', 'State'], axis=1, inplace=True)

# Rename columns
dmast_df.columns = ['state_FIPS', 'deaths', 'population', 'crude', 'crude_CI_lower', 'crude_CI_upper', 'crude_se',
                    'age_adj', 'age_adj_CI_lower', 'age_adj_CI_upper', 'age_adj_se', 'pct_deaths']

# Drop AK & HI
if DROP_AKHI:
    dmast_df = dmast_df[dmast_df.state_FIPS != 2]
    dmast_df = dmast_df[dmast_df.state_FIPS != 15]

# Set FIPS as index
dmast_df.set_index(['state_FIPS'], inplace=True)

# Drop unneeded columns (incl. CI, SE, & percent)
dmast_df.drop(['population', 'crude_CI_lower', 'crude_CI_upper', 'crude_se',
               'age_adj_CI_lower', 'age_adj_CI_upper', 'age_adj_se', 'pct_deaths'], axis=1, inplace=True)

# Rename columns
dmast_df.columns = ['DM_deaths_all', 'DM_crude_rate_all', 'DM_age_adj_rate_all']


state_dfs['dm_all'] = dmast_df
# dmast_df

In [20]:
# Drop state FIPS
if 'state_FIPS' in dmd_df:
    dmd_df.drop(['state_FIPS'], axis=1, inplace=True)
if 'state_FIPS' in dma_df:
    dma_df.drop(['state_FIPS'], axis=1, inplace=True)

# Merge UCOD tables
dm_df = dmd_df.join(dma_df)

dfs['dm'] = dm_df
# dm_df

Data from CDC WONDER, Underlying Cause of Death Compressed Mortality database:
* grouped by county
* including 95% CI and SE on the crude rate, age adjusted rate (+ 95% CI & SE), and percent of total deaths
* all states, ages, genders, races, ethnicities, years (1999-2015), days, places, etc
* ICD-10 Codes X42, X62, and Y12 ONLY 
* including totals, zero values, & suppressed values, 2 decimal places of precision

https://wonder.cdc.gov/controller/saved/D132/D16F416

Also, the same, but:
* all causes of death

https://wonder.cdc.gov/controller/saved/D132/D16F719

Also, the same, but:
* grouped by state

https://wonder.cdc.gov/controller/saved/D132/D16F417

Also, the same, but:
* grouped by state
* all causes of death

https://wonder.cdc.gov/controller/saved/D132/D16F720

To retrieve the data:
1. go the the URL
2. click "I Agree"
3. click "Send"
4. rename the files "cm_county_opiates.tsv", "cm_county_all.tsv", "cm_state_opiates.tsv", and "cm_state_all.tsv",
   respectively

In [21]:
df_file = "cm_county_opiates.tsv"
cmo_df = pd.read_table(df_file, na_values=['Suppressed', 'Unreliable'], skipfooter=158, engine='python')

# Drop first two cols
cmo_df.drop(['Notes', 'County'], axis=1, inplace=True)

# Rename columns
cmo_df.columns = ['FIPS', 'deaths', 'population', 'crude', 'crude_CI_lower', 'crude_CI_upper', 'crude_se',
                  'age_adj', 'age_adj_CI_lower', 'age_adj_CI_upper', 'age_adj_se', 'pct_deaths']

# Remove "(Unreliable)"
cmo_df.crude = cmo_df.crude.apply(lambda r: float(r.split()[0]) if type(r) is str else r)
cmo_df.age_adj = cmo_df.age_adj.apply(lambda r: float(r.split()[0]) if type(r) is str else r)

# Convert percentage to [0, 1]
cmo_df.pct_deaths = cmo_df.pct_deaths.apply(lambda p: (float(p.replace('%', '')) if type(p) is str else p)/100)

# Add the state FIPS
cmo_df['state_FIPS'] = cmo_df.FIPS // 1000

# Drop AK & HI
if DROP_AKHI:
    cmo_df = cmo_df[cmo_df.state_FIPS != 2]
    cmo_df = cmo_df[cmo_df.state_FIPS != 15]

# Set FIPS as index
cmo_df.set_index(['FIPS'], inplace=True)

# Drop unneeded columns (incl. CI, SE, & percent)
cmo_df.drop(['population', 'crude_CI_lower', 'crude_CI_upper', 'crude_se',
             'age_adj_CI_lower', 'age_adj_CI_upper', 'age_adj_se', 'pct_deaths'], axis=1, inplace=True)

# Rename columns
cmo_df.columns = ['CM_deaths_opiates', 'CM_crude_rate_opiates', 'CM_age_adj_rate_opiates', 'state_FIPS']

# dfs['cm_opiates'] = cmo_df
# cmo_df

In [22]:
df_file = "cm_county_all.tsv"
cma_df = pd.read_table(df_file, na_values=['Suppressed', 'Unreliable'], skipfooter=148, engine='python')

# Drop first two cols
cma_df.drop(['Notes', 'County'], axis=1, inplace=True)

# Rename columns
cma_df.columns = ['FIPS', 'deaths', 'population', 'crude', 'crude_CI_lower', 'crude_CI_upper', 'crude_se',
                   'age_adj', 'age_adj_CI_lower', 'age_adj_CI_upper', 'age_adj_se']

# Remove "(Unreliable)"
cma_df.crude = cma_df.crude.apply(lambda r: float(r.split()[0]) if type(r) is str else r)
cma_df.age_adj = cma_df.age_adj.apply(lambda r: float(r.split()[0]) if type(r) is str else r)

# Add the state FIPS
cma_df['state_FIPS'] = cma_df.FIPS // 1000

# Drop AK & HI
if DROP_AKHI:
    cma_df = cma_df[cma_df.state_FIPS != 2]
    cma_df = cma_df[cma_df.state_FIPS != 15]

# Set FIPS as index
cma_df.set_index(['FIPS'], inplace=True)

# Drop unneeded columns (incl. CI, SE, & percent)
cma_df.drop(['population', 'crude_CI_lower', 'crude_CI_upper', 'crude_se',
             'age_adj_CI_lower', 'age_adj_CI_upper', 'age_adj_se'], axis=1, inplace=True)

# Rename columns
cma_df.columns = ['CM_deaths_all', 'CM_crude_rate_all', 'CM_age_adj_rate_all', 'state_FIPS']

# dfs['cm_all'] = cma_df
# cma_df

In [23]:
df_file = "cm_state_opiates.tsv"
cmost_df = pd.read_table(df_file, na_values=['Suppressed', 'Unreliable'], skipfooter=63, engine='python')

# Drop first two cols
cmost_df.drop(['Notes', 'State'], axis=1, inplace=True)

# Rename columns
cmost_df.columns = ['state_FIPS', 'deaths', 'population', 'crude', 'crude_CI_lower', 'crude_CI_upper', 'crude_se',
                   'age_adj', 'age_adj_CI_lower', 'age_adj_CI_upper', 'age_adj_se', 'pct_deaths']

# Remove "(Unreliable)"
cmost_df.crude = cmost_df.crude.apply(lambda r: float(r.split()[0]) if type(r) is str else r)
cmost_df.age_adj = cmost_df.age_adj.apply(lambda r: float(r.split()[0]) if type(r) is str else r)

# Convert percentage to [0, 1]
cmost_df.pct_deaths = cmost_df.pct_deaths.apply(lambda p: (float(p.replace('%', '')) if type(p) is str else p)/100)

# Drop AK & HI
if DROP_AKHI:
    cmost_df = cmost_df[cmost_df.state_FIPS != 2]
    cmost_df = cmost_df[cmost_df.state_FIPS != 15]

# Set FIPS as index
cmost_df.set_index(['state_FIPS'], inplace=True)

# Drop unneeded columns (incl. CI, SE, & percent)
cmost_df.drop(['population', 'crude_CI_lower', 'crude_CI_upper', 'crude_se',
               'age_adj_CI_lower', 'age_adj_CI_upper', 'age_adj_se', 'pct_deaths'], axis=1, inplace=True)

# Rename columns
cmost_df.columns = ['CM_deaths_opiates', 'CM_crude_rate_opiates', 'CM_age_adj_rate_opiates']

# state_dfs['cm'] = cmost_df
# cmost_df

In [24]:
df_file = "cm_state_all.tsv"
cmast_df = pd.read_table(df_file, na_values=['Suppressed', 'Unreliable'], skipfooter=48, engine='python')

# Drop first two cols
cmast_df.drop(['Notes', 'State'], axis=1, inplace=True)

# Rename columns
cmast_df.columns = ['state_FIPS', 'deaths', 'population', 'crude', 'crude_CI_lower', 'crude_CI_upper', 'crude_se',
                   'age_adj', 'age_adj_CI_lower', 'age_adj_CI_upper', 'age_adj_se', 'pct_deaths']

# Remove "(Unreliable)"
cmast_df.crude = cmast_df.crude.apply(lambda r: float(r.split()[0]) if type(r) is str else r)
cmast_df.age_adj = cmast_df.age_adj.apply(lambda r: float(r.split()[0]) if type(r) is str else r)

# Convert percentage to [0, 1]
cmast_df.pct_deaths = cmast_df.pct_deaths.apply(lambda p: (float(p.replace('%', '')) if type(p) is str else p)/100)

# Drop AK & HI
if DROP_AKHI:
    cmast_df = cmast_df[cmast_df.state_FIPS != 2]
    cmast_df = cmast_df[cmast_df.state_FIPS != 15]

# Set FIPS as index
cmast_df.set_index(['state_FIPS'], inplace=True)

# Drop unneeded columns (incl. CI, SE, & percent)
cmast_df.drop(['population', 'crude_CI_lower', 'crude_CI_upper', 'crude_se',
               'age_adj_CI_lower', 'age_adj_CI_upper', 'age_adj_se', 'pct_deaths'], axis=1, inplace=True)

# Rename columns
cmast_df.columns = ['CM_deaths_all', 'CM_crude_rate_all', 'CM_age_adj_rate_all']

# state_dfs['cm'] = cmast_df
# cmast_df

In [25]:
# Drop state FIPS
if 'state_FIPS' in cmo_df:
    cmo_df.drop(['state_FIPS'], axis=1, inplace=True)
if 'state_FIPS' in cma_df:
    cma_df.drop(['state_FIPS'], axis=1, inplace=True)

# Merge CM tables
cm_df = cmo_df.join(cma_df)

dfs['cm'] = cm_df
# cm_df

**2015 SAHIE data**

Codebook: https://www.census.gov/did/www/sahie/data/files/SAHIE-File-Layout-Overview.pdf

In [26]:
df_file = "sahie_2015.csv"
if not os.path.isfile(df_file):
    if not os.path.isfile("sahie_2015.csv.zip"):
        ur.urlretrieve("https://www.census.gov/did/www/sahie/data/files/sahie_2015.csv.zip",
                       "sahie_2015.csv.zip")
        with zipfile.ZipFile("sahie_2015.csv.zip", "r") as zip_ref:
            zip_ref.extractall('.')
temp_files.append("sahie_2015.csv.zip")

sahie_df = pd.read_csv(df_file, skiprows=79, low_memory=False)
temp_files.append(df_file)

# Drop non-county data
sahie_df = sahie_df[sahie_df.geocat == 50]

# Create FIPS column
sahie_df['FIPS'] = sahie_df.statefips*1000 + sahie_df.countyfips 

# Drop AK & HI
if DROP_AKHI:
    sahie_df = sahie_df[sahie_df.statefips != 2]
    sahie_df = sahie_df[sahie_df.statefips != 15]

# Drop all the rows with specified (non-zero) age, race, sex, income
for col in ['agecat', 'racecat', 'sexcat', 'iprcat']:
    sahie_df = sahie_df[sahie_df[col] == 0]
    
# Drop unneeded columns
sahie_df.drop(['year', 'version', 'statefips', 'countyfips', 'geocat', 'agecat', 'racecat', 'sexcat',
               'iprcat', 'nipr_moe', 'state_name', 'county_name', 'Unnamed: 25'], axis=1, inplace=True)

for col in ['NIPR', 'nui_moe', 'nic_moe']:
    sahie_df[col] = sahie_df[col].astype(float)

for col in ['NUI', 'NIC']:
    sahie_df[col] = sahie_df[col].astype(int)
    
# Convert percentages to range [0,1]
# for col in sahie_df.columns:
#     if 'pct' in col or 'PCT' in col:
#         sahie_df[col] = sahie_df[col].astype(float)
#         sahie_df[col] /= 100

# Set index to FIPS
sahie_df.set_index(['FIPS'], inplace=True)

# Drop MOEs
sahie_df.drop(['NIPR', 'nui_moe', 'nic_moe', 'pctui_moe', 'pctic_moe', 'pctelig_moe', 'pctliic_moe'],
              axis=1, inplace=True)
# Drop percentages by category
sahie_df.drop(['PCTUI', 'PCTIC'], axis=1, inplace=True)

# Rename columns
sahie_df.columns = ['uninsured', 'insured', 'pct_uninsured', 'pct_insured']


dfs['sahie'] = sahie_df
# sahie_df

**Medicare Part D Opioid Prescribing data** from CMS.gov

In [27]:
df_file = "Part_D_Opioid_Prescribing_Change_Geographic_2013_2014.xlsx"
if not os.path.isfile(df_file):
    if not os.path.isfile("Part_D_Opioid_Geographic_Data.zip"):
        ur.urlretrieve("https://www.cms.gov/Research-Statistics-Data-and-Systems/Statistics-Trends-and-Reports/Medicare-Provider-Charge-Data/Downloads/Part_D_Opioid_Geographic_Data.zip",
                       "Part_D_Opioid_Geographic_Data.zip")
        with zipfile.ZipFile("Part_D_Opioid_Geographic_Data.zip", "r") as zip_ref:
            zip_ref.extractall('.')
temp_files.append("Part_D_Opioid_Geographic_Data.zip")

opioid_df = pd.read_excel(df_file, skiprows=3, sheetname="COUNTY")
temp_files.append(df_file)

# Rename columns
opioid_df.columns = ['state', 'state_abbr', 'state_fips', 'county', 'FIPS', 'partD_prescribers_2013',
                     'opioid_claims_2013', 'overall_claims_2013', 'opioid_rate_2013', 'partD_prescribers_2014',
                     'opioid_claims_2014', 'overall_claims_2014', 'opioid_rate_2014', 'pct_diff']

# Drop AK & HI
if DROP_AKHI:
    opioid_df = opioid_df[opioid_df['state_fips'] != 2]
    opioid_df = opioid_df[opioid_df['state_fips'] != 15]

# Set index to FIPS
opioid_df.set_index(['FIPS'], inplace=True)

# Drop unneeded columns
opioid_df.drop(['state_abbr', 'state_fips', 'county', 'state'], axis=1, inplace=True)

# Convert percentages to [0, 1]:
for col in ['opioid_rate_2013', 'opioid_rate_2014', 'pct_diff']:
    opioid_df[col] /= 100

# Drop unneeded columns (percentages)
# opioid_df.drop(['opioid_rate_2013', 'opioid_rate_2014', 'pct_diff'], axis=1, inplace=True)

# Rename columns
opioid_df.columns = ['partd_prescribers_2013', 'partd_claims_opioid_2013',
                     'partd_claims_all_2013', 'partd_opioid_rate_2013',
                     'partd_prescribers_2014', 'partd_claims_opioid_2014',
                     'partd_claims_all_2014', 'partd_opioid_rate_2014',
                     'partd_rate_diff']


dfs['opioid'] = opioid_df
# opioid_df

**IRS data**

In [28]:
df_file = "irs_2014.xls"
if not os.path.isfile(df_file):
    ur.urlretrieve("https://www.irs.gov/pub/irs-soi/14incyall.xls",
                   "irs_2014.xls")

irs_df = pd.read_excel(df_file, skiprows=6, header=None, skip_footer=18)
temp_files.append(df_file)

# Rename columns
irs_df.columns = ['state_fips', 'state', 'county_fips', 'county', 'returns', 'single', 'joint', 'headofhousehold',
                  'paid_prep', 'exemptions', 'dependents', 'vol_prep', 'vita_prep', 'tce_prep', 'AGI',
                  'all_inc_num', 'all_inc_amt', 'salaries_num', 'salaries_amt', 'interest_num', 'interest_amt',
                  'ord_div_num', 'ord_div_amt', 'qual_div_num', 'qual_div_amt', 'state_loc_ref_num',
                  'state_loc_ref_amt', 'bus_num', 'bus_amt', 'capgain_num', 'capgain_amt', 'indivret_num',
                  'indivret_amt', 'pension_num', 'pension_amt', 'farm_num', 'unemp_num', 'unemp_amt', 'ss_num',
                  'ss_amt', 'scorp_num', 'scorp_amt', 'stat_num', 'stat_amt', 'ed_num', 'ed_amt', 'se_num',
                  'se_amt', 'sehi_num', 'sehi_amt', 'ira_num', 'ira_amt', 'sloan_num', 'sloan_amt', 'tuition_num',
                  'tuition_amt', 'prod_num', 'prod_amt', 'item_num', 'item_amt', 'item_amt_agi', 'state_loc_num',
                  'state_loc_amt', 'sales_num', 'sales_amt', 're_num', 're_amt', 'taxes_num', 'taxes_amt',
                  'mortgage_num', 'mortgage_amt', 'contr_num', 'contr_amt', 'taxable_num', 'taxable_amt',
                  'tax_bef_cred_num', 'tax_bef_cred_amt', 'amt_num', 'amt_amt', 'exaptc_num', 'exaptc_amt',
                  'tax_cred_num', 'tax_cred_amt', 'foreign_num', 'foreign_amt', 'child_num', 'child_amt',
                  'ed_cred_num', 'ed_cred_amt', 'ret_sav_num', 'ret_sav_amt', 'ctc_num', 'ctc_amt', 'energy_num',
                  'energy_amt', 'set_num', 'set_amt', 'ptc_num', 'ptc_amt', 'aptc_num', 'aptc_amt', 'hcir_num',
                  'hcir_amt', 'ttp_num', 'ttp_amt', 'eic_num', 'eic_amt', 'exeic_num', 'exeic_amt', 'actc_num',
                  'actc_amt', 'red_num', 'red_amt', 'net_ptc_num', 'net_ptc_amt', 'ic_num', 'ic_amt', 'tl_num',
                  'tl_amt', 'medicare_num', 'medicare_amt', 'net_iit_num', 'net_iit_amt', 'due_num', 'due_amt',
                  'refund_num', 'refund_amt']


# Drop AK & HI
if DROP_AKHI:
    irs_df = irs_df[irs_df.state_fips != 2]
    irs_df = irs_df[irs_df.state_fips != 15]

# Drop non-county data
irs_df = irs_df[irs_df.county_fips != 0]

# Create FIPS column
irs_df['FIPS'] = irs_df.state_fips*1000 + irs_df.county_fips 

# Drop unneeded columns
irs_df.drop(['state', 'county', 'state_fips', 'county_fips'], axis=1, inplace=True)

# Set index to FIPS
irs_df.set_index(['FIPS'], inplace=True)

# Rename columns
irs_df.columns = ['IRS_'+col for col in irs_df.columns]

dfs['irs'] = irs_df
# irs_df

** USDA Employment data **

In [29]:
df_file = "Unemployment.xls"
if not os.path.isfile(df_file):
    if not os.path.isfile(df_file):
        ur.urlretrieve("https://www.ers.usda.gov/webdocs/DataFiles/48747/"+df_file,
                       df_file)
temp_files.append(df_file)

emp_df = pd.read_excel(df_file, skiprows=9, sheetname="Unemployment Med HH Inc")
temp_files.append(df_file)

# Drop AK & HI
if DROP_AKHI:
    emp_df = emp_df[emp_df['State'] != 'AK']
    emp_df = emp_df[emp_df['State'] != 'HI']

# Drop PR
emp_df = emp_df[emp_df['State'] != 'PR']

# Lowercase column names
emp_df.columns = [_.lower() for _ in emp_df.columns]

# Drop state-level data
emp_df = emp_df[emp_df.fipstxt % 1000 != 0]

# Set index to FIPS
emp_df['FIPS'] = emp_df.fipstxt
emp_df.set_index(['FIPS'], inplace=True)

# Drop unneeded columns
emp_df.drop(['rural_urban_continuum_code_2013', 'urban_influence_code_2013', 'metro_2013', 'state', 'area_name',
            'fipstxt'], axis=1, inplace=True)

# Drop rates
emp_df.drop([_ for _ in emp_df.columns if 'rate' in _], axis=1, inplace=True)

# Convert to int
for col in emp_df.columns:
    emp_df[col] = emp_df[col].astype(int)

dfs['emp'] = emp_df
# emp_df

** USDA Education data **

In [30]:
df_file = "Education.xls"
if not os.path.isfile(df_file):
    if not os.path.isfile(df_file):
        ur.urlretrieve("https://www.ers.usda.gov/webdocs/DataFiles/48747/"+df_file,
                       df_file)
temp_files.append(df_file)
edu_df = pd.read_excel(df_file, skiprows=4)

edu_df.columns = ['FIPS', 'state', 'drop_0', 'drop_1', 'drop_2', 'drop_3', 'drop_4',
                  'ed_no_hs_diploma_1970', 'ed_hs_diploma_1970', 'ed_some_college_1970', 'ed_college_1970',
                  'ed_pct_no_hs_diploma_1970', 'ed_pct_hs_diploma_1970',
                  'ed_pct_some_college_1970', 'ed_pct_college_1970',
                  'ed_no_hs_diploma_1980', 'ed_hs_diploma_1980', 'ed_some_college_1980', 'ed_college_1980',
                  'ed_pct_no_hs_diploma_1980', 'ed_pct_hs_diploma_1980',
                  'ed_pct_some_college_1980', 'ed_pct_college_1980',
                  'ed_no_hs_diploma_1990', 'ed_hs_diploma_1990', 'ed_some_college_1990', 'ed_college_1990',
                  'ed_pct_no_hs_diploma_1990', 'ed_pct_hs_diploma_1990',
                  'ed_pct_some_college_1990', 'ed_pct_college_1990',
                  'ed_no_hs_diploma_2000', 'ed_hs_diploma_2000', 'ed_some_college_2000', 'ed_college_2000',
                  'ed_pct_no_hs_diploma_2000', 'ed_pct_hs_diploma_2000',
                  'ed_pct_some_college_2000', 'ed_pct_college_2000',
                  'ed_no_hs_diploma_2015', 'ed_hs_diploma_2015', 'ed_some_college_2015', 'ed_college_2015',
                  'ed_pct_no_hs_diploma_2015', 'ed_pct_hs_diploma_2015',
                  'ed_pct_some_college_2015', 'ed_pct_college_2015']

# Drop AK & HI
if DROP_AKHI:
    edu_df = edu_df[edu_df['state'] != 'AK']
    edu_df = edu_df[edu_df['state'] != 'HI']

# Drop PR & US
edu_df = edu_df[edu_df['state'] != 'US']
edu_df = edu_df[edu_df['state'] != 'PR']

# Drop state-level data
edu_df = edu_df[edu_df.FIPS % 1000 != 0]

# Set index to FIPS
edu_df.set_index(['FIPS'], inplace=True)

# Drop unneeded columns
edu_df.drop(['state'] + [_ for _ in edu_df.columns if 'drop' in _], axis=1, inplace=True)

# Fill NAs with 0 and convert to int or [0, 1] percentages
edu_df.fillna(0, inplace=True)
for col in edu_df.columns:
    if '_pct_' in col:
        edu_df[col] = edu_df[col]/100
    else:
        edu_df[col] = edu_df[col].astype(int)

dfs['edu'] = edu_df
# edu_df

** 2014 Social Capital Variables ** from PennState's College of Agricultural Sciences

In [31]:
df_file = "social_capital.xls"
if not os.path.isfile(df_file):
    if not os.path.isfile(df_file):
        ur.urlretrieve("http://aese.psu.edu/nercrd/community/social-capital-resources/social-capital-variables-for-2014/social-capital-variables-spreadsheet-for-2014/at_download/file",
                       df_file)
temp_files.append(df_file)
sc_df = pd.read_excel(df_file)

sc_df.columns = ['FIPS', 'religious_orgs', 'civic_assns', 'businesses_assns', 'political_orgs', 'prof_orgs',
                  'labor_orgs', 'bowling_ctrs', 'rec_ctrs', 'golf_courses', 'sports_clubs',
                  'pop', 'assns_agg', 'voter_turnout', 'census_resp', 'npos', 'social_capital']

# Drop AK & HI
if DROP_AKHI:
    sc_df = sc_df[sc_df['FIPS'] // 1000 != 2]
    sc_df = sc_df[sc_df['FIPS'] // 1000 != 15]

# Set index to FIPS
sc_df.set_index(['FIPS'], inplace=True)

# Drop unneeded columns
sc_df.drop(['pop', 'assns_agg', 'voter_turnout', 'census_resp', 'npos'], axis=1, inplace=True)


dfs['sc'] = sc_df
# sc_df

** 2010 Estimated Use of Water in the United States**

In [32]:
df_file = "water.xlsx"
if not os.path.isfile(df_file):
    if not os.path.isfile(df_file):
        ur.urlretrieve("https://water.usgs.gov/watuse/data/2010/usco2010.xlsx",
                       df_file)
temp_files.append(df_file)
water_df = pd.read_excel(df_file)

# Drop AK & HI
if DROP_AKHI:
    water_df = water_df[water_df['STATE'] != 'AK']
    water_df = water_df[water_df['STATE'] != 'HI']

# Drop PR & VI:
for state in ['PR', 'VI']:
    water_df = water_df[water_df['STATE'] != state]

# Set index to FIPS
water_df.set_index(['FIPS'], inplace=True)

# Drop unneeded columns
water_df.drop(['STATE', 'STATEFIPS', 'COUNTY', 'COUNTYFIPS', 'YEAR', 'TP-TotPop'], axis=1, inplace=True)

water_df.columns = ['water_'+c.replace('-', '_') for c in water_df.columns]


dfs['water'] = water_df
# water_df

** 2012 Uniform Crime Reporting Program data from ICPSR **

Download [Arrest](http://www.icpsr.umich.edu/cgi-bin/bob/terms2?study=35019&ds=1&bundle=delimited&path=NACJD) and [Crime Report](http://www.icpsr.umich.edu/cgi-bin/bob/terms2?study=35019&ds=4&bundle=delimited&path=NACJD) data from ICPSR (which requires agreeing to terms).

Open each zip and place the TSV files in ICPSR_35019/DS0001 and ICPSR_35019/DS0004, respectively, into the county-data directory where this notebook is located.

In [33]:
df_file = "35019-0001-Data.tsv"
arrests_df = pd.read_table(df_file, engine='python')

arrests_df['FIPS'] = arrests_df.FIPS_ST*1000 + arrests_df.FIPS_CTY

# Drop AK & HI
if DROP_AKHI:
    arrests_df = arrests_df[arrests_df.FIPS_ST != 2]
    arrests_df = arrests_df[arrests_df.FIPS_ST != 15]

# Set FIPS as index
arrests_df.set_index(['FIPS'], inplace=True)

# Drop unneeded cols
arrests_df.drop(['STUDYNO', 'EDITION', 'PART', 'IDNO', 'CPOPARST', 'FIPS_ST', 'FIPS_CTY'], axis=1, inplace=True)

# Rename columns
arrests_df.columns = ['agencies', 'multi_jurisdiction', 'coverage', 'arrests', 'p1_total', 'p1_violent',
                      'p1_property', 'murder', 'rape', 'robbery', 'agassault', 'burglary', 'larcenty', 'mv_theft',
                      'arson', 'assault_other', 'forgery', 'fraud', 'embezzlement', 'stolen_prop', 'vandalism',
                      'weapons', 'prostitution', 'sex_off', 'drug_total', 'drug_sale', 'cocaine_sale',
                      'marijuana_sale', 'synthetic_sale', 'drug_other_sale', 'drug_poss', 'cocaine_poss',
                      'marijuana_poss', 'synthetic_poss', 'drug_other_poss', 'gambling', 'bookmaking', 'numbers',
                      'gambling_other', 'off_fam_child', 'dui', 'liquor', 'drunkenness', 'disorderly', 'vagrancy',
                      'nontraffic_other', 'suspicion', 'curfew', 'runaway']
arrests_df.columns = ['UCRP_'+_ for _ in arrests_df.columns]

# dfs['arrests'] = arrests_df
# arrests_df

In [34]:
df_file = "35019-0004-Data.tsv"
reports_df = pd.read_table(df_file, engine='python')

reports_df['FIPS'] = reports_df.FIPS_ST*1000 + reports_df.FIPS_CTY

# Drop AK & HI
if DROP_AKHI:
    reports_df = reports_df[reports_df.FIPS_ST != 2]
    reports_df = reports_df[reports_df.FIPS_ST != 15]

# Set FIPS as index
reports_df.set_index(['FIPS'], inplace=True)

# Drop unneeded cols
reports_df.drop(['STUDYNO', 'EDITION', 'PART', 'IDNO', 'CPOPARST', 'CPOPCRIM', 'FIPS_ST', 'FIPS_CTY', 'AG_ARRST',
                 'AG_OFF', 'COVIND'], axis=1, inplace=True)

# Rename columns
reports_df.columns = ['violent', 'property', 'murder', 'rape', 'robbery', 'agassault', 'burglary', 'larceny',
                      'mv_theft', 'arson']
reports_df.columns = ['UCRP_'+_+'_rep' for _ in reports_df.columns]

# dfs['arrests'] = reports_df
# reports_df

In [35]:
ucrp_df = arrests_df.join(reports_df)
dfs['ucrp'] = ucrp_df
# ucrp_df

More data to consider integrating:

* https://www.census.gov/support/USACdataDownloads.html
* https://catalog.data.gov/dataset/county-level-data-sets
* https://www.huduser.gov/portal/datasets/nsp_foreclosure_data.html


for fn in set(temp_files):
    if os.path.isfile(fn):
        os.unlink(fn)

In [36]:
list(dfs.keys())

['name',
 'gaz',
 'natural',
 'rucc',
 'uic',
 'election',
 'census',
 'rcms',
 'oasdi_ben',
 'oasdi_pay',
 'saipe',
 'ssi',
 'vha',
 'dm',
 'cm',
 'sahie',
 'opioid',
 'irs',
 'emp',
 'edu',
 'sc',
 'water',
 'ucrp']

In [37]:
# df_list = ['name', 'gaz', 'natural', 'election', 'census', 'oasdi_ben', 'oasdi_pay', 'saipe', 'ssi', 'dm', 'cm',
#            'sahie', 'opioid', 'irs', 'emp', 'edu']
df_list = list(dfs.keys())

master = dfs[df_list[0]]
for df in df_list[1:]:
    master = master.join(dfs[df], how='left', rsuffix=df)
master

,state,county,housing_units,area_land,area_water,lat,long,census_div,jan_temp_mean,jan_sun_mean,...,UCRP_violent_rep,UCRP_property_rep,UCRP_murder_rep,UCRP_rape_rep,UCRP_robbery_rep,UCRP_agassault_rep,UCRP_burglary_rep,UCRP_larceny_rep,UCRP_mv_theft_rep,UCRP_arson_rep
FIPS,,,,,,,,,,,,,,,,,,,,,
1001,AL,Autauga County,22135,594.436,9.952,32.536382,-86.644490,6.0,47.400,130.0,...,139,1765,3,15,34,87,447,1233,85,108
1003,AL,Baldwin County,104061,1589.784,437.527,30.659218,-87.746067,6.0,51.900,152.0,...,446,4988,8,30,76,332,967,3829,192,31
1005,AL,Barbour County,11829,884.876,19.639,31.870670,-85.405456,6.0,48.800,152.0,...,48,474,0,4,8,36,90,362,21,0
1007,AL,Bibb County,8981,622.582,3.587,33.015893,-87.127148,6.0,44.800,130.0,...,49,399,1,4,8,36,122,251,27,0
1009,AL,Blount County,23887,644.776,5.852,33.977448,-86.567246,6.0,41.900,130.0,...,122,1347,1,11,9,101,397,865,86,9
1011,AL,Bullock County,4493,622.805,2.338,32.101759,-85.717261,6.0,47.800,149.0,...,56,156,0,2,12,43,98,55,3,23
1013,AL,Butler County,9964,776.829,1.053,31.751667,-86.681969,6.0,49.200,152.0,...,109,621,6,5,11,87,182,408,30,3
1015,AL,Calhoun County,53289,605.868,6.419,33.771706,-85.822513,6.0,42.800,130.0,...,678,5339,6,37,131,504,1710,3482,147,34
1017,AL,Chambers County,17004,596.531,6.582,32.917943,-85.391812,6.0,44.200,149.0,...,192,1345,1,7,24,160,374,875,96,5


In [38]:
master.to_csv('county-data.csv')